In [1]:
import requests
import datetime
import re
import MySQLdb

In [2]:
# 爬取原始数据
def getDataFromWeb(day):
    urls = ["http://www.czce.com.cn/portal/DFSStaticFiles/Future/{0}/{1}/FutureDataDaily.txt", 
            "http://www.czce.com.cn/portal/exchange/{0}/datadaily/{1}.txt"]
    for url in urls:
        url = url.format(day.year, day.strftime('%Y%m%d'))
        r = requests.get(url)
        if r.status_code==200:
            return r.text
    return ''

In [3]:
# 处理原始数据
def process(rawdata):
    data = []
    rawdata = rawdata.split('\r\n')
    reg = re.compile(r'[A-Z]{2}\d{3}')
    for line in rawdata:
        # 将数据处理成列表
        line = line.split('|')
        if len(line)==1:
            line = line[0]
            line = line.split(',')
        line = [x.strip() for x in line]
        line = [re.sub(',','',x) for x in line]
        # 筛掉非合约行
        if not reg.match(line[0]):
            continue
        code = line[0][:2].lower()
        deli = '1' + line[0][2:]
        _open = float(line[2])
        high = float(line[3])
        low = float(line[4])
        close = float(line[5])
        settle = float(line[6])
        volume = int(float(line[9]))
        oi = int(float(line[10]))
        turnover = float(line[12])
        data.append([code, deli, settle, _open, close, high, low, volume, oi, turnover])
    return data

In [4]:
# 插入数据库
def load(day, data):
    conn = MySQLdb.connect(host='127.0.0.1', user='root', passwd='codegeass', db='future', charset='utf8')
    cursor = conn.cursor()
    clean_sql = "delete from contract_daily where date=%s and code=%s and delivery=%s"
    insert_sql = 'insert into contract_daily (date,code,delivery,settle,open,close,high,low,volume,oi,turnover) values (' + '%s,'*10 + '%s' + ')'
    for row in data:
        # 清理旧数据
        cursor.execute(clean_sql, tuple([day]+row[:2]))
        conn.commit()
        # 插入新数据
        cursor.execute(insert_sql, tuple([day]+row))
        conn.commit()
    cursor.close()
    conn.close()

In [8]:
day = datetime.date(2017,1,1)
end = datetime.date(2017,11,10)
step = datetime.timedelta(1)
while day<=end:
    if day.isoweekday not in [6,7]:
        rawdata = getDataFromWeb(day)
        if rawdata:
            data = process(rawdata)
            load(day,data)
    print day.isoformat()
    day += step

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17
2017-01-18
2017-01-19
2017-01-20
2017-01-21
2017-01-22
2017-01-23
2017-01-24
2017-01-25
2017-01-26
2017-01-27
2017-01-28
2017-01-29
2017-01-30
2017-01-31
2017-02-01
2017-02-02
2017-02-03
2017-02-04
2017-02-05
2017-02-06
2017-02-07
2017-02-08
2017-02-09
2017-02-10
2017-02-11
2017-02-12
2017-02-13
2017-02-14
2017-02-15
2017-02-16
2017-02-17
2017-02-18
2017-02-19
2017-02-20
2017-02-21
2017-02-22
2017-02-23
2017-02-24
2017-02-25
2017-02-26
2017-02-27
2017-02-28
2017-03-01
2017-03-02
2017-03-03
2017-03-04
2017-03-05
2017-03-06
2017-03-07
2017-03-08
2017-03-09
2017-03-10
2017-03-11
2017-03-12
2017-03-13
2017-03-14
2017-03-15
2017-03-16
2017-03-17
2017-03-18
2017-03-19
2017-03-20
2017-03-21
2017-03-22
2017-03-23
2017-03-24
2017-03-25
2017-03-26
2017-03-27
2017-03-28
2017-03-29
2017-03-30
2017-03-31
2017-04-01